In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
import random
import time
import itertools
import numpy as np
import spacy
import pandas as pd

In [2]:
nlp = spacy.load('en')

In [3]:
driver = webdriver.Chrome(executable_path=r'chromedriver.exe')

In [5]:
driver.get('http://www.mlb.com')
normal_delay = random.normalvariate(2, 0.5)
time.sleep(normal_delay)
# print('Sleeping for {} seconds'.format(normal_delay))

In [6]:
'''
Question 1:
'''
stats_header_bar = driver.find_element_by_class_name('megamenu-static-navbar__menu-item--stats')
stats_header_bar.click()
time.sleep(normal_delay)

In [7]:
# click into team section
team_bar = driver.find_element_by_id('st_parent')
team_bar.click()
time.sleep(normal_delay)

In [8]:
# find hitting season element to select
hitting_season_element = driver.find_element_by_id('st_hitting_season')
#select season 
season_select = Select(hitting_season_element)

In [9]:
season_select.select_by_value('2015')
time.sleep(normal_delay)

In [10]:
# find regular season of 2015
RS_game_type = driver.find_element_by_xpath("//select[@name='st_hitting_game_type']/option[text()='Regular Season']")
RS_game_type.click()
time.sleep(normal_delay)

In [11]:
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC

In [12]:
ques_one_page_source = driver.page_source

In [ ]:
soup = BeautifulSoup(ques_one_page_source, 'html5lib')

In [14]:
# find table div {id: datagrid}
table_div = soup.find('div', attrs = {'id': 'datagrid'})

In [15]:
# find table in table_div
table = table_div.find('table', attrs ={'class': 'stats_table'})

In [16]:
# find table body
table_body = table.find('tbody')

In [17]:
# find table head
table_head = table.find('thead')

In [18]:
ths = table_head.find_all('th')

In [19]:
# find hr score th & team name th
hr_head = ths[10]
team_name = ths[1]

In [20]:
hr_head.text

'HR▼'

In [21]:
n_rows = 0
hr_score_list = []
team_name_list = []
column_names = []
column_names.extend((team_name.text,hr_head.text))
for tr in table_body.find_all('tr'):
    # find all tds in each tr
    tds = tr.find_all('td')
    # retrieve name tds
    team_name = tds[1].text
    team_name_list.append(team_name)
    # hr score of each row(team)
    hr_score = tds[10].text
    if len(hr_score) > 0:
        n_rows = n_rows + 1
        hr_score_list.append(hr_score)
# df = pd.DataFrame(columns = column_names, data = hr_score_list, index= range(1,n_rows+1))

In [22]:
# retrieve max score & index of it
max_score = max(hr_score_list)
hr_score_list.append(max_score)
max_index = hr_score_list.index(max_score)
# add max_score_team_name to list
team_name_list.append(team_name_list[max_index])
# combine two lists
ques_one_list = list(zip(team_name_list,hr_score_list))

In [23]:
# create panda df
df = pd.DataFrame(columns = column_names, data = ques_one_list, index= range(1,n_rows+2))

In [24]:
len_of_index = len(df.index)
# change last row name to 'Most HRs'
df.rename(index={len_of_index: 'Most HRs'})

,Team,HR▼
1,Detroit Tigers,151
2,Toronto Blue Jays,232
3,Kansas City Royals,139
4,San Francisco Giants,136
5,Colorado Rockies,186
6,Boston Red Sox,161
7,Arizona Diamondbacks,154
8,Miami Marlins,120
9,Pittsburgh Pirates,140
10,Texas Rangers,172


In [25]:
df.to_csv('ques_one.csv', sep='\t')

In [26]:
'''question one:
which team had most HRs in regular season in 2015? print full team name'''
print(ques_one_list[-1])

('Toronto Blue Jays', '232')


In [27]:
'''
Question 2:
'''
field_sets_div = driver.find_element_by_id('st_hitting-0')

In [28]:
field_sets = field_sets_div.find_element_by_class_name('widget-radio')

In [29]:
field_sets_spans = field_sets.find_elements_by_class_name('ui-button-text')

In [30]:
MLB_tab = field_sets_spans[0]
AL_tab = field_sets_spans[1]
NL_tab = field_sets_spans[2]

In [31]:
AL_tab.click()
time.sleep(normal_delay)

In [32]:
ques_two_page_source = driver.page_source
AL_soup = BeautifulSoup(ques_two_page_source, 'html.parser')

In [33]:
AL_table_grid = AL_soup.find('div', attrs={'id': 'datagrid'})

In [34]:
# find table in table_div
AL_table = AL_table_grid.find('table',attrs={'class':'data_grid'})

In [35]:
# get AL team list
AL_team_list = []
team_col_name = AL_table.thead.find_all('th')[1].text
# tbody of table
AL_table_body = AL_table.tbody
# retrieve name from each tr
for tr in AL_table_body.find_all('tr'):
    tds = tr.find_all('td')
    AL_team_list.append(tds[1].text)

In [36]:
# Similarly get NL_team_list
NL_tab.click() #click into NL_tab page
time.sleep(normal_delay)

In [37]:
NL_soup = BeautifulSoup(driver.page_source, 'html.parser')
# NL table grid & table
NL_table_grid = NL_soup.find('div', attrs={'id': 'datagrid'})
# find table in table_div
NL_table = NL_table_grid.find('table',attrs={'class':'data_grid'})
# get AL team list
NL_team_list = []
team_col_name = NL_table.thead.find_all('th')[1].text
# tbody of table
NL_table_body = NL_table.tbody
# retrieve name from each tr
for tr in NL_table_body.find_all('tr'):
    tds = tr.find_all('td')
    NL_team_list.append(tds[1].text)

In [38]:
ques_one_name_list = [i[0] for i in ques_one_list][:-1]
AL_list = []
NL_list = []
for name in ques_one_name_list:
    # index of name in list
    index = ques_one_name_list.index(name)
    if name in AL_team_list:
        AL_list.append(ques_one_list[index])
    elif name in NL_team_list:
        NL_list.append(ques_one_list[index])

In [39]:
AL_hr_list = [int(i[1]) for i in AL_list]
NL_hr_list = [int(i[1]) for i in NL_list]
# calculate average of both league
AL_ave_hr = sum(AL_hr_list) / len(AL_hr_list)
NL_ave_hr = sum(NL_hr_list) / len(NL_hr_list)
ques_two_a = []
if NL_ave_hr > AL_ave_hr:
    ques_two_a = ['NL', 'AVE HRs: ' + str(NL_ave_hr)]
elif AL_ave_hr > NL_ave_hr:
    ques_two_a = ['AL', 'AVE HRs: ' + str(AL_ave_hr)]
else:
    ques_two_a = ['Same number for both league: ' + str(NL_ave_hr)]

In [40]:
'''
Question two a
'''
ques_two_a

['AL', 'AVE HRs: 175.6']

In [41]:
# find MLB tab and click
MLB_tab.click()
time.sleep(normal_delay)

In [42]:
# spot first inning page and click into the page
hitting_split = driver.find_element_by_id('st_hitting_hitting_splits')
split_select = Select(hitting_split)
split_select.select_by_value('i01')
time.sleep(normal_delay)

In [43]:
inning_soup = BeautifulSoup(driver.page_source, 'html.parser')
inning_table_div = inning_soup.find('div', attrs={'id':'datagrid'})
inning_table = inning_table_div.find('table',attrs={'class':'data_grid'})

In [44]:
def first_inning_ave_hr(table):
    first_inning_AL_list = []
    first_inning_NL_list = []
    first_inning_Alleague_HR = []
    first_inning_Alleague_list = []
    ques_two_b_list = []
    # team name & hr cols in first inning
    col_names = []
    ths = table.thead.find_all('th')
    going_to_add_colnames = [ths[1].text,ths[10].abbr.text]
    col_names.extend(going_to_add_colnames)
    # put first inning hrs into [first_inning_Alleague_HR]
    for tr in table.tbody.find_all('tr'):
        tds = tr.find_all('td')
        ques_two_b_list.append(tds[1].text)
        first_inning_Alleague_HR.append(tds[10].text)
    # combine name & hr
    first_inning_Alleague_list = list(zip(ques_two_b_list, first_inning_Alleague_HR))
    # convert it to dataframe
    first_inning_Alleague_df = pd.DataFrame(columns = col_names, data = first_inning_Alleague_list, index= range(1,n_rows+1))
    first_inning_Alleague_df.to_csv('ques_two_b.csv', sep="\t")
    print(ques_two_b_list)
    for name in ques_two_b_list:
        index = ques_two_b_list.index(name)
        # if name in AL_team_list, then put corresponding hr to first_inning_AL_list
        if name in AL_team_list:
            first_inning_AL_list.append(int(first_inning_Alleague_HR[index]))
        # if name in NL_team_list, then put corresponding hr to first_inning_NL_list
        elif name in NL_team_list:
            first_inning_NL_list.append(int(first_inning_Alleague_HR[index]))
            
    return first_inning_AL_list, first_inning_NL_list, first_inning_Alleague_df

In [45]:
first_inning_AL_hr, first_inning_NL_hr, first_inning_df = first_inning_ave_hr(inning_table)

['Colorado Rockies', 'Kansas City Royals', 'Detroit Tigers', 'Texas Rangers', 'New York Yankees', 'Arizona Diamondbacks', 'Seattle Mariners', 'Miami Marlins', 'Milwaukee Brewers', 'San Francisco Giants', 'Cincinnati Reds', 'Washington Nationals', 'Atlanta Braves', 'Minnesota Twins', 'Pittsburgh Pirates', 'Houston Astros', 'Toronto Blue Jays', 'Boston Red Sox', 'Baltimore Orioles', 'Tampa Bay Rays', 'St. Louis Cardinals', 'Oakland Athletics', 'Cleveland Indians', 'Los Angeles Dodgers', 'Chicago White Sox', 'New York Mets', 'Los Angeles Angels', 'San Diego Padres', 'Philadelphia Phillies', 'Chicago Cubs']


In [46]:
'''
Question two b:
'''
# calculate the ave hrs for both league
first_inning_AL_ave = sum(first_inning_AL_hr) / len(first_inning_AL_hr)
first_inning_NL_ave = sum(first_inning_NL_hr) / len(first_inning_NL_hr)
ques_two_b= []
if first_inning_AL_ave > first_inning_NL_ave:
    ques_two_b = ['AL', 'First inning ave HRs: ' + str(first_inning_AL_ave)]
elif first_inning_NL_ave > first_inning_AL_ave:
    ques_two_b = ['NL', 'First inning ave HRs: ' + str(first_inning_NL_ave)]
else:
    ques_two_b = ['Same number for both league: ', str(first_inning_NL_ave)]

In [47]:
first_inning_AL_ave

23.666666666666668

In [48]:
ques_two_a

['AL', 'AVE HRs: 175.6']

In [49]:
'''
Question 3:
'''
# click player tab from team tab
player_tab = driver.find_element_by_id('sp_parent')
player_tab.click()
time.sleep(normal_delay)

In [50]:
# change year from 15 to 17
hitting_season_element = driver.find_element_by_id('sp_hitting_season')
#select season 
season_select = Select(hitting_season_element)
season_select.select_by_value('2017')
time.sleep(normal_delay)

In [51]:
# find New York Yankees
team_id_tab = driver.find_element_by_id('sp_hitting_team_id')
team_select = Select(team_id_tab)
team_select.select_by_value('147')
time.sleep(normal_delay)

In [52]:
select_split = driver.find_element_by_xpath("//select[@name='sp_hitting_hitting_splits']/option[text()='Select Split']")
select_split.click()
time.sleep(normal_delay)

In [53]:
ques_three_page_source = driver.page_source
ques_three_soup = BeautifulSoup(ques_three_page_source, 'html.parser')

In [54]:
# find table_div for question 3
yankees_table_div = ques_three_soup.find('div', attrs={'id': 'datagrid'})
yankees_table = yankees_table_div.find('table', attrs={'class': 'data_grid'})

In [55]:
def ques_three_fun(table):
    col_names = []
    # table header & body
    table_header = yankees_table.thead
    table_body = yankees_table.tbody
    # player_name, Pos, AB, AVG
    heads = table_header.find_all('th')
    header_player = heads[1].text
    header_pos = heads[5].text
    header_AB = heads[7].abbr.text
    header_AVG = heads[18].abbr.text
    # col_names
    going_to_add_cols = [header_player,header_pos,header_AB,header_AVG]
    col_names.extend(going_to_add_cols)
    
    #intialize cols list
    player_list = []
    pos_list = []
    ab_list = []
    avg_list = []
    trs = table_body.find_all('tr')
    for tr in trs:
        tds = tr.find_all('td')
        # get name, pos, ab, avg value
        player_name = tds[1].text
        player_list.append(player_name)
        pos = tds[5].text
        pos_list.append(pos)
        AB = tds[7].text
        ab_list.append(AB)
        AVG = tds[18].text
        avg_list.append(AVG)
    data = list(zip(player_list, pos_list, ab_list, avg_list))
    ques_three_df = pd.DataFrame(columns = col_names, data = data, index= range(1,len(trs)+1))
    ques_three_df.to_csv('ques_three.csv', sep='\t')
    return ques_three_df

In [56]:
ques_three_df = ques_three_fun(yankees_table)

In [57]:
# retrieve AB col
# player_col = ques_three_df['Player']
# pos_col = ques_three_df['Pos'] 
AB_col = ques_three_df['AB']
# avg_col = ques_three_df['AVG']
q3_best_player = [ques_three_df.loc[ques_three_df.index[31]]][0]
for index,value in enumerate(AB_col):
    if(int(value)>=30):
        if ques_three_df.loc[ques_three_df.index[index], 'AVG'] > q3_best_player[3]:
#             put row into list to store
            q3_best_player=[ques_three_df.loc[ques_three_df.index[index]]][0]
        
#         q3_best_player = [ques_three_df.loc[ques_three_df.index[index], 'Player'], ques_three_df.loc[ques_three_df.index[index], 'Pos'], ques_three_df.loc[ques_three_df.index[index], 'AVG']]
'''Question 3a answer'''
q3_best_player = q3_best_player[0:2]

In [58]:
# use numpy logical or to grab player whose pos are RF,LF,CF
ques_three_b_df = ques_three_df[np.logical_or(np.logical_or(ques_three_df['Pos'] == 'CF' ,ques_three_df['Pos'] == 'LF'), ques_three_df['Pos'] == 'RF')]

In [59]:
'''Question 3b answer'''
yankees_best_outfielder = ques_three_b_df.loc[ques_three_b_df.index[0]][0:2]

In [60]:
'''
Question 4
'''
# find hitting season element to select
hitting_season_element = driver.find_element_by_id('sp_hitting_season')
#select season 
season_select = Select(hitting_season_element)
season_select.select_by_value('2015')
time.sleep(normal_delay)

In [61]:
All_team_select = driver.find_element_by_xpath("//select[@name='sp_hitting_team_id']/option[text()='All Teams']")
All_team_select.click()
time.sleep(normal_delay)

In [62]:
field_sets_div = driver.find_element_by_id('sp_hitting-1')
field_sets = field_sets_div.find_element_by_class_name('widget-radio')
field_sets_spans = field_sets.find_elements_by_class_name('ui-button-text')
AL_tab = field_sets_spans[1]
AL_tab.click()
time.sleep(normal_delay)

In [63]:
ques_four_table_div = driver.find_element_by_id('datagrid')
ques_four_table = ques_four_table_div.find_element_by_class_name('data_grid')
AB_tab = ques_four_table.find_element_by_tag_name('thead').find_element_by_class_name('dg-ab')
AB_tab.click()
time.sleep(normal_delay)

In [64]:
ques_four_page_source = driver.page_source
ques_four_soup = BeautifulSoup(ques_four_page_source, 'html5lib')
ques_four_soup_div = ques_four_soup.find('div', attrs={'id':'datagrid'})
ques_four_soup_table = ques_four_soup_div.find('table', attrs={'class': 'data_grid'})
ques_four_target_player = ques_four_soup_table.tbody.find_all('tr')[0]
# Player name, team name, position
tds = ques_four_target_player.find_all('td')
ques_four_player_name = tds[1].text
ques_four_team_name = tds[2].text
ques_four_pos = tds[5].text
'''
Question 4 asnwer
'''
ques_four_player = [ques_four_player_name, ques_four_team_name, ques_four_pos]

In [65]:
hitting_season_element = driver.find_element_by_id('sp_hitting_season')
#select season 
season_select = Select(hitting_season_element)
season_select.select_by_value('2014')
time.sleep(normal_delay)

In [66]:
# all-start game tab
All_start_game_type = driver.find_element_by_xpath("//select[@name='sp_hitting_game_type']/option[text()='All-Star Game']")
All_start_game_type.click()
time.sleep(normal_delay)
# AL->MLB
field_sets_spans = driver.find_element_by_id('sp_hitting-1').find_element_by_class_name('widget-radio').find_elements_by_class_name('ui-button-text')
# field_sets = field_sets_div.find_element_by_class_name('widget-radio')
# field_sets_spans = field_sets.find_elements_by_class_name('ui-button-text')
MLB_tab = field_sets_spans[0]
MLB_tab.click()
time.sleep(normal_delay)

In [67]:
Latin_America = ['Argentina', 
                 'Bolivia', 
                 'Brazil',
                 'Chile',
                 'Colombia',
                 'Costa Rica',
                 'Cuba',
                 'Dominican Republic',
                 'Ecuador',
                 'El Salvador',
                 'French Guiana',
                 'Guadeloupe',
                 'Guatemala',
                 'Haiti',
                 'Honduras',
                 'Martinique',
                 'Mexico',
                 'Nicaragua',
                 'Panama',
                 'Paraguay',
                 'Peru',
                 'Puerto Rico',
                 'Saint Barthélemy',
                 'Saint Martin',
                 'Uruguay',
                 'Venezuela']

In [68]:
# Question five table body
ques_five_page_source = driver.page_source
ques_five_table_body = BeautifulSoup(ques_five_page_source, 'html5lib').find('div', attrs={'id':'datagrid'}).find('table', attrs={'class': 'data_grid'}).tbody
# link list
player_link_list = []
ques_five_base_link = 'http://www.mlb.com'
for tr in ques_five_table_body.find_all('tr'):
#     td for each player
    tds = tr.find_all('td')
#     second column(name col)
    name_col = tds[1]
    # find a href link
    link = name_col.find('a', href=True)
    player_link_list.append(ques_five_base_link + link['href'])

In [69]:
def get_player_info(link, full_name_list, team_name, league_name):
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    section = soup.body.find('section', {'id': 'quick-stats'})
    div = section.find('div', {'class': 'player-bio'})
    spans = div.find_all('span', {'class': 'label'})
    full_name = div.find('li', {'class': 'full-name'}).text
    team_table = soup.body.find('div', {'id': 'careerStats'}).find('div', {'class': 'responsive-datatable__pinned'}).find('table')
    trs = team_table.tbody.findAll('tr')
    t_name = ''
    l_name = ''
    for tr in trs:
        tds = tr.findAll('td')
        if tds[0].find('span') and tds[0].find('span').text == '2014':
            t_name = tds[1].text.strip()
            l_name = tds[2].text.strip()
    for index,span in enumerate(spans):
        # spot span with text == Born:
        if span.text == 'Born:':
            info = div.find_all('li')[index].text
            # retrieve last word in str
#             last_word = info.split()[-1]
            doc = nlp(info.strip())
            for ent in doc.ents:
                if ent.label_ == 'GPE' and ent.text in Latin_America:
                    full_name_list.append(full_name)
                    team_name.append(t_name)
                    league_name.append(l_name)
#     data = list(zip(full_name_list, team_name, league_name))
    return full_name_list, team_name, league_name

In [71]:
full_name_list = []
team_name = []
league_name = []
for link in player_link_list:
    get_player_info(link, full_name_list, team_name, league_name)
ques_5_data = list(zip(full_name_list, team_name, league_name))
col_name = ['Player Name', 'Team Name', 'League Name']

In [72]:
ques_five_df = pd.DataFrame(columns = col_name, data = ques_5_data, index= range(1,len(full_name_list)+1))
ques_five_df.to_csv('ques_five.csv', sep='\t')
'''
Question 5
'''
ques_five_df

,Player Name,Team Name,League Name
1,Full Name: Jose Miguel Cabrera,DET,AL
2,Full Name: Yasiel Puig,LAD,NL
3,Full Name: Aramis Nin Ramirez,MIL,NL
4,Full Name: José Antonio Bautista,TOR,AL
5,Full Name: Robinson Jose Canó,SEA,AL
6,Full Name: Yoenis Céspedes,2 teams,-
7,Full Name: Nelson Ramon Cruz,BAL,AL
8,Full Name: Carlos Argelis Gómez,MIL,NL
9,Full Name: Alexei Fernando Ramirez,CWS,AL
10,Full Name: José Dariel Abreu,CWS,AL


In [73]:
'''
Question 6
'''
########### Python 3.2 #############
import http.client, urllib.request, urllib.parse, urllib.error, base64, json, os

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': '1d7ff7298e7f4c84802a0ef013940c01',
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('api.fantasydata.net')
    conn.request("GET", "/v3/mlb/stats/JSON/Games/2016", "{body}", headers)
    response = conn.getresponse()
    data = json.loads(response.read())
    print(type(data))
#     print(data)
    with open('Question_6_schedule.txt', 'w') as outfile:
        outfile.write(json.dumps(data))
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))


<class 'list'>


In [74]:
# read file into json
with open("Question_6_schedule.txt") as inputfile:
    response = json.load(inputfile)
    # Filter python objects with list comprehensions
response = [x for x in response if x['AwayTeam']=='HOU' or x['HomeTeam'] == 'HOU']
output_json = json.dumps(response,indent=4, sort_keys=True)
ques_6_col_names = ['Opponent team names', 'Game date', 'stadiumID']
#initialize opponent team names, game date, stadiumID list
opponent_team_names = []
game_date = []
stadiumID = []
for game in response:
    game_date.append(game['DateTime'])
    stadiumID.append(game['StadiumID'])
    if game['AwayTeam'] == 'HOU':
        opponent_team_names.append(game['HomeTeam'])
    elif game['HomeTeam'] == 'HOU':
        opponent_team_names.append(game['AwayTeam'])

In [75]:
# get stadium data 
try:
    conn = http.client.HTTPSConnection('api.fantasydata.net')
    conn.request("GET", "/v3/mlb/stats/JSON/Stadiums?", "{body}", headers)
    response = conn.getresponse()
    data = json.loads(response.read())
    with open('Question_6_stadium.txt', 'w') as outfile:
        outfile.write(json.dumps(data))
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [76]:
with open('Question_6_stadium.txt') as inputfile:
    stadiums = json.load(inputfile)
required_stadium_info = []
required_stadium_info = [[x['StadiumID'], x['Name'], x['City'], x['State']] for x in stadiums]

In [77]:
for index,ID in enumerate(stadiumID):
    for ind,info in enumerate(required_stadium_info):
        if ID == info[0]:
            stadiumID[index] = required_stadium_info[ind][1:4]


In [78]:
# get team name data 
try:
    conn = http.client.HTTPSConnection('api.fantasydata.net')
    conn.request("GET", "/v3/mlb/stats/JSON/AllTeams?", "{body}", headers)
    response = conn.getresponse()
    data = json.loads(response.read())
    with open('Question_6_team_name.txt', 'w') as outfile:
        outfile.write(json.dumps(data))
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [80]:
with open('Question_6_team_name.txt') as inputfile:
    ques_6_team_names = json.load(inputfile)
new_team_list = []
# find name col in ques 6 df and replace name over key
for inedx,name in enumerate(opponent_team_names):
    for ind,team_info in enumerate(ques_6_team_names):
        if name == team_info['Key']:
            new_team_list.append(team_info['Name'])
ques_6_data = list(zip(new_team_list, game_date, stadiumID))
ques_6_df = pd.DataFrame(columns= ques_6_col_names, data = ques_6_data, index =range(1, len(new_team_list)+1))
ques_6_df.to_csv('Question_6_final.csv', sep="\t")
'''
Question 6 answer from API schedule, team name, stats
'''